### 1. 기본 import 등

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
train = pd.read_csv('./open/train.csv')

# year, month, item_id 기준으로 value 합산 (seq만 다르다면 value 합산)
monthly = (
    train
    .groupby(["item_id", "year", "month"], as_index=False)["value"]
    .sum()
)

# year, month를 하나의 키(ym)로 묶기
monthly["ym"] = pd.to_datetime(
    monthly["year"].astype(str) + "-" + monthly["month"].astype(str).str.zfill(2)
)

# item_id × ym 피벗 (월별 총 무역량 매트릭스 생성)
pivot = (
    monthly
    .pivot(index="item_id", columns="ym", values="value")
    .fillna(0.0)
)

pivot.head()

ym,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,...,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01,2025-03-01,2025-04-01,2025-05-01,2025-06-01,2025-07-01
item_id,,,,,,,,,,,,,,,,,,,,,
AANGBULD,14276.0,52347.0,53549.0,0.0,26997.0,84489.0,0.0,0.0,0.0,0.0,...,428725.0,144248.0,26507.0,25691.0,25805.0,0.0,38441.0,0.0,441275.0,533478.0
AHMDUILJ,242705.0,120847.0,197317.0,126142.0,71730.0,149138.0,186617.0,169995.0,140547.0,89292.0,...,123085.0,143451.0,78649.0,125098.0,80404.0,157401.0,115509.0,127473.0,89479.0,101317.0
ANWUJOKX,0.0,0.0,0.0,63580.0,81670.0,26424.0,8470.0,0.0,0.0,80475.0,...,0.0,0.0,0.0,27980.0,0.0,0.0,0.0,0.0,0.0,0.0
APQGTRMF,383999.0,512813.0,217064.0,470398.0,539873.0,582317.0,759980.0,216019.0,537693.0,205326.0,...,683581.0,2147.0,0.0,25013.0,77.0,20741.0,2403.0,3543.0,32430.0,40608.0
ATLDMDBO,143097177.0,103568323.0,118403737.0,121873741.0,115024617.0,65716075.0,146216818.0,97552978.0,72341427.0,87454167.0,...,60276050.0,30160198.0,42613728.0,64451013.0,38667429.0,29354408.0,42450439.0,37136720.0,32181798.0,57090235.0


### 2. function define

In [ ]:
### correlation 관련 함수

def safe_corr(x, y):
    if np.std(x) == 0 or np.std(y) == 0:
        return 0.0
    return float(np.corrcoef(x, y)[0, 1])

def find_comovement_pairs(pivot, max_lag=6, min_nonzero=12, corr_threshold=0.55):
    items = pivot.index.to_list()
    months = pivot.columns.to_list()
    n_months = len(months)

    results = []

    for i, leader in tqdm(enumerate(items)):
        x = pivot.loc[leader].values.astype(float)
        if np.count_nonzero(x) < min_nonzero:
            continue

        for follower in items:
            if follower == leader:
                continue

            y = pivot.loc[follower].values.astype(float)
            if np.count_nonzero(y) < min_nonzero:
                continue

            best_lag = None
            best_corr = 0.0

            # lag = 1 ~ max_lag 탐색
            for lag in range(1, max_lag + 1):
                if n_months <= lag:
                    continue
                corr = safe_corr(x[:-lag], y[lag:])
                if abs(corr) > abs(best_corr):
                    best_corr = corr
                    best_lag = lag

            # 임계값 이상이면 공행성쌍으로 채택
            if best_lag is not None and abs(best_corr) >= corr_threshold:
                results.append({
                    "leading_item_id": leader,
                    "following_item_id": follower,
                    "best_lag": best_lag,
                    "max_corr": best_corr,
                })

    pairs = pd.DataFrame(results)
    return pairs

In [17]:
### build train data
### diff 사용 version

def build_training_data(pivot, pairs):
    """
    공행성쌍 + 시계열을 이용해 (X, y) 학습 데이터를 만드는 함수
    input X:
      - b_t, b_t_1, a_t_lag, max_corr, best_lag
    target y:
      - b_t_plus_1
    """
    months = pivot.columns.to_list()
    n_months = len(months)

    rows = []

    for row in pairs.itertuples(index=False):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag_base = int(row.best_lag_base)
        corr_base = float(row.max_corr_base)
        lag_diff = int(row.best_lag_diff)
        corr_diff = float(row.max_corr_diff)

        if leader not in pivot.index or follower not in pivot.index:
            continue

        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        # t+1이 존재하고, t-lag >= 0인 구간만 학습에 사용
        for t in range(max(lag_base, 1), n_months - 1):
            b_t = b_series[t]
            b_t_1 = b_series[t - 1]
            a_t_lag = a_series[t - lag_base]
            b_t_plus_1 = b_series[t + 1]

            rows.append({
                "b_t": b_t,
                "b_t_1": b_t_1,
                "a_t_lag": a_t_lag,
                "max_corr": corr_base,
                "best_lag": float(lag_base),
                "target": b_t_plus_1,
            })

    df_train = pd.DataFrame(rows)
    return df_train

In [18]:
### predict and submission making

def predict(pivot, pairs, reg):
    months = pivot.columns.to_list()
    n_months = len(months)

    # 가장 마지막 두 달 index (2025-7, 2025-6)
    t_last = n_months - 1
    t_prev = n_months - 2

    preds = []

    for row in tqdm(pairs.itertuples(index=False)):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag_base = int(row.best_lag_base)
        corr_base = float(row.max_corr_base)
        lag_diff = int(row.best_lag_diff)
        corr_diff = float(row.max_corr_diff)   

        if leader not in pivot.index or follower not in pivot.index:
            continue

        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        # t_last - lag 가 0 이상인 경우만 예측
        if t_last - lag_base < 0:
            continue

        b_t = b_series[t_last]
        b_t_1 = b_series[t_prev]
        a_t_lag = a_series[t_last - lag_base]

        X_test = np.array([[b_t, b_t_1, a_t_lag, corr_base, float(lag_base)]])
        y_pred = reg.predict(X_test)[0]

        # (후처리 1) 음수 예측 → 0으로 변환
        # (후처리 2) 소수점 → 정수 변환 (무역량은 정수 단위)
        y_pred = max(0.0, float(y_pred))
        y_pred = int(round(y_pred))

        preds.append({
            "leading_item_id": leader,
            "following_item_id": follower,
            "value": y_pred,
        })

    df_pred = pd.DataFrame(preds)
    return df_pred

### 3. main

In [23]:
raw_pairs = find_comovement_pairs(pivot, min_nonzero=8, max_lag=10,corr_threshold=0.325)

print(f"raw pair 개수 : {len(raw_pairs)}")

diff_pivot = pivot.diff(axis=1).iloc[:, 1:] 

diff_pairs = find_comovement_pairs(diff_pivot, min_nonzero=4, max_lag=10,corr_threshold=0.325)

print(f"diff pair 개수 : {len(diff_pairs)}")

pairs = pd.merge(
        raw_pairs,
        diff_pairs,
        on=["leading_item_id", "following_item_id"],
        suffixes=("_base", "_diff")
    )

print(f"pair 개수 : {len(pairs)}")

100it [00:14,  6.91it/s]
1it [00:00,  5.78it/s]

raw pair 개수 : 4079


100it [00:16,  6.19it/s]

diff pair 개수 : 5341
pair 개수 : 3017


In [24]:
pairs = pairs[pairs.apply(lambda x: abs(x.best_lag_base - x.best_lag_diff) < 2, axis=1)].reset_index(drop=True)

In [26]:
df_train_model = build_training_data(pivot, pairs)

feature_cols = ['b_t', 'b_t_1', 'a_t_lag', 'max_corr', 'best_lag']

train_X = df_train_model[feature_cols].values
train_y = df_train_model["target"].values

model = RandomForestRegressor(max_depth=6, 
                              n_estimators=200,
                              random_state=3141592)

model.fit(train_X,train_y)

RandomForestRegressor(max_depth=6, n_estimators=200, random_state=3141592)

In [27]:
submission = predict(pivot, pairs, model)
submission.to_csv('./submission/diff_trial2.csv', index=False)

2011it [00:21, 92.31it/s] 


In [28]:
len(submission)

2011

In [ ]:
def custom_grid(thresholds, lags, nonzeros):

    custom_grid.thresh = 0
    custom_grid.lag = 0
    custom_grid.nz = 0
    custom_grid.nrmse = -10e10
    custom_grid.best = None

    for th,lag,nz in product(thresholds, lags, nonzeros):
        pairs = find_comovement_pairs(pivot, min_nonzero=nz, max_lag=lag,corr_threshold=th)

        val_idx = np.random.choice(pairs.index, size = int(len(pairs)*0.2), replace=False)

        val_pairs = pairs.loc[val_idx]
        train_pairs = pairs.drop(val_idx)

        df_train = build_training_data(pivot, train_pairs)
        df_val = build_training_data(pivot, val_pairs)

        feature_cols = ['b_t', 'b_t_1', 'a_t_lag', 'max_corr', 'best_lag']

        train_X = df_train[feature_cols].values
        train_y = df_train["target"].values

        val_X = df_val[feature_cols].values
        val_y = df_val["target"].values

        model = RandomForestRegressor(random_state=3141592)

        param_grid = {
            'n_estimators' : [200,400],
            "max_depth" : [3,6],
            # "ccp_alpha": [0.0, 1e-3,],
            # "max_features" : ['sqrt','log2']
        }

        grid_search = GridSearchCV(
            estimator= model,
            param_grid= param_grid,
            cv = 5,
            scoring = 'neg_root_mean_squared_error',
        )

        grid_search.fit(train_X,train_y)

        if custom_grid.nrmse < grid_search.best_score_:
            custom_grid.nrmse = grid_search.best_score_
            custom_grid.thresh = th
            custom_grid.lag = lag
            custom_grid.nz = nz
            custom_grid.best = grid_search.best_estimator_

    return custom_grid.thresh, custom_grid.lag, custom_grid.nz,custom_grid.best



In [7]:
thresholds = [0.325, 0.35 ,0.375]
lags = [6,10]
nzs = [8]

th,lag,nz,model = custom_grid(thresholds,lags,nzs)

100it [00:11,  8.58it/s]
100it [00:14,  7.12it/s]
100it [00:08, 11.86it/s]
100it [00:13,  7.26it/s]
100it [00:08, 11.73it/s]
100it [00:13,  7.36it/s]


In [8]:
pairs = find_comovement_pairs(pivot, min_nonzero=nz, max_lag=lag,corr_threshold=th)

df_train_model = build_training_data(pivot, pairs)

feature_cols = ['b_t', 'b_t_1', 'a_t_lag', 'max_corr', 'best_lag']

train_X = df_train_model[feature_cols].values
train_y = df_train_model["target"].values

model.fit(train_X,train_y)


100it [00:13,  7.25it/s]


RandomForestRegressor(max_depth=6, n_estimators=200, random_state=3141592)

In [9]:
submission = predict(pivot, pairs, model)
submission.to_csv('./submission/base_tunning_1.csv', index=False)

0it [00:00, ?it/s]

3416it [00:36, 94.15it/s]


In [10]:
print(th,lag, nz)

0.35 10 8
